In [1]:
import os
import polars as pl
import polars_ds as pds
import lightgbm as lgbm
import xgboost as xgb

In [2]:
plan_train = pl.scan_parquet("data/train.parquet")
plan_test = pl.scan_parquet("data/test.parquet")

In [3]:
plan_test.head().collect()

row_id,date_id,time_id,symbol_id,weight,is_scored,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,…,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
i64,i16,i16,i8,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,0,0,3.169998,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
1,0,0,1,2.165993,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0
2,0,0,2,3.06555,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,-0.0,-0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
3,0,0,3,2.698642,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
4,0,0,4,1.80333,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0


In [4]:
plan_train.select(pl.col("date_id").unique().count() * 0.75).collect()

date_id
f64
1274.25


In [5]:
plan_train_set = plan_train.filter(pl.col("date_id") <= 1275)
plan_valid_set = plan_train.filter(pl.col("date_id") > 1275)
plan_test_set = plan_test.filter(pl.col("date_id") > 1275)

In [6]:
plan_train_feats = plan_train_set.select(pl.col("^feature_.*$"))
plan_train_feats.collect().head()

feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,null,0.28118,0.269163,0.349028,…,null,-0.181716,null,null,null,0.564021,2.088506,0.832022,null,0.204797,null,null,-0.808103,null,-2.037683,0.727661,null,-0.989118,-0.345213,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498
null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,null,-1.512286,-1.414357,-1.823322,…,null,null,null,null,null,-10.835207,-0.002704,-0.621836,null,1.172836,null,null,-1.625862,null,-1.410017,1.063013,null,0.888355,0.467994,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441
null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,null,-0.320921,-0.95809,-2.436589,…,null,null,null,null,null,-1.420632,-3.515137,-4.67776,null,0.535897,null,null,-0.72542,null,-2.29417,1.764551,null,-0.120789,-0.063458,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792
null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,null,0.544138,-0.087091,-1.500147,…,null,null,null,null,null,0.382074,2.669135,0.611711,null,2.413415,null,null,1.313203,null,-0.810125,2.939022,null,3.988801,1.834661,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244
null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,null,-0.08842,-0.995003,-2.635336,…,null,null,null,null,null,-2.0146,-2.321076,-3.711265,null,1.253902,null,null,0.476195,null,-0.771732,2.843421,null,1.379815,0.411827,-1.36224,null,null,null,null,null,-0.948601,-0.136814,-0.447704,-1.141761,0.099631,-0.661928,null,null,3.678076,2.793581,2.61825,3.418133


In [7]:
plan_valid_feats = plan_valid_set.select(pl.col("^feature_.*$"))
plan_valid_feats.collect().head()

feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0.653638,2.198754,-0.198297,0.224291,2.852725,0.825279,2.266075,0.766185,0.036033,11,7,76,-0.777789,0.209351,-0.543527,null,-0.327792,null,-0.974575,-1.833984,-1.155611,-0.129055,-0.18312,0.781583,-0.170369,-0.272233,1.377188,0.839882,0.140909,-0.222573,-0.279321,-0.20658,null,null,-1.610952,-2.160539,0.891483,…,null,-0.639679,null,-1.486683,0.940282,0.068512,0.545302,0.118213,null,0.865484,null,null,-0.392299,null,-1.330438,2.243315,null,0.168773,-0.062025,0.61328,-0.386102,-0.16773,-0.268163,-1.372035,-1.248691,-0.827321,0.056538,-0.44961,-0.891529,0.354493,-0.723254,null,null,-0.215982,-0.143597,-0.291303,-0.369795
0.60227,2.037193,-0.205099,0.164635,2.089605,1.046435,3.319236,1.683911,0.065478,11,7,76,-0.592238,0.952971,-0.23747,null,-0.079895,null,-1.218711,-1.262983,-1.69775,0.105305,0.983402,1.797576,-0.909877,-1.239271,0.589516,1.175475,1.095303,-0.55926,-0.517902,0.071531,null,null,1.337161,1.07081,1.137291,…,null,-0.811337,null,-1.576211,0.714847,0.562134,-0.288386,0.144059,null,1.210798,null,null,0.123525,null,-0.870009,2.13797,null,0.251116,0.107858,0.61328,-0.363495,-0.243295,-0.367433,-1.938556,-2.104432,-0.765297,0.929756,-0.132269,-0.707898,0.643057,-0.336551,null,null,-0.265186,-0.178259,-0.232595,-0.312367
0.07485,2.167284,-0.164738,0.12676,2.52651,0.49116,1.783477,0.911501,0.04823,81,2,59,-0.826606,0.345691,-0.805456,null,-0.459432,null,-1.652119,-2.089218,0.277807,-0.212113,-1.062779,-1.349508,1.317141,-0.122495,0.966619,1.045411,0.556983,-0.466345,-0.543724,-0.260239,null,null,1.08513,1.173192,2.015734,…,null,0.109272,null,-1.986501,1.218621,1.268623,-0.68015,0.045226,null,0.045898,null,null,-1.315452,null,-2.11331,1.412617,null,-1.162586,-0.590398,0.61328,-0.41582,-0.262088,-0.389611,-1.130174,-1.156497,-0.978691,-0.100609,-0.39627,-0.987192,0.528615,-0.380612,null,null,-0.191881,-0.138189,-0.302623,-0.251448
0.590885,2.073502,0.029229,0.6459,2.263874,0.454036,1.212922,0.716108,0.033077,4,3,11,-1.120522,-0.016231,-0.694629,null,-0.781894,null,-2.180358,-1.498456,0.225393,0.239456,-0.986674,-1.295119,1.924675,0.475872,-0.42378,-2.050294,-1.516776,-0.783469,-0.742045,0.326289,null,null,-0.601088,-1.137856,-1.111369,…,null,1.10189,null,-1.317938,1.696318,0.069499,1.282036,0.41416,null,1.386339,null,null,0.781272,null,-1.206338,2.72017,null,1.766053,1.007932,0.61328,-0.436664,-0.237977,-0.320159,-2.211359,-2.286742,-0.722229,-0.110039,-0.626917,-0.589325,0.08313,-0.474187,null,null,2.34694,4.054324,4.119953,4.731141
0.603547,2.226605,-0.089176,0.267814,2.414924,0.594974,1.683337,1.430113,0.041524,15,1,9,-1.016874,0.288751,-0.417176,null,-0.532268,null,-1.28309,-1.495769,-0.745321,-0.041418,-1.167268,-1.00016,0.501984,-0.962245,-0.951838,-1.55216,-0.54972,-0.716685,-0.83996,-0.034939,null,null,1.459631,1.729872,2.505692,…,null,0.056481,null,-1.372009,0.800305,0.686966,0.384807,0.416568,null,0.488048,null,null,-0.494901,

In [8]:
plan_train_target = plan_train_set.select(pl.col("responder_6"))
plan_train_target.collect().head()

responder_6
f32
0.775981
0.703665
2.109352
1.114137
-3.57282


In [9]:
plan_valid_target = plan_valid_set.select(pl.col("responder_6"))
plan_valid_target.collect().head()

responder_6
f32
-0.803032
-1.076396
-1.088245
0.001118
-0.69548


In [10]:
plan_train_weight = plan_train_set.select(pl.col("weight"))
plan_train_weight.collect().head()

weight
f32
3.889038
1.370613
2.285698
0.690606
0.44057


In [11]:
plan_valid_weight = plan_valid_set.select(pl.col("weight"))
plan_valid_weight.collect().head()

weight
f32
1.772753
3.322994
1.490064
1.252261
0.989003


In [12]:
dtrain = xgb.DMatrix(
    plan_train_feats.collect(),
    label=plan_train_target.collect(),
    weight=plan_train_weight.collect(),
    feature_names=plan_train_feats.collect_schema().names(),
)
dvalid = xgb.DMatrix(
    plan_valid_feats.collect(),
    label=plan_valid_target.collect(),
    weight=plan_valid_weight.collect(),
    feature_names=plan_valid_feats.collect_schema().names(),
)


In [21]:
# model = xgb.XGBRegressor(objective="reg:squarederror")
param = {
    "objective": "reg:squarederror",
    "max_depth": 5,
    "learning_rate": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "nthread": 32,
    "verbosity": 1,
}

num_boost_round = 500
model = xgb.train(
    param, dtrain, num_boost_round, evals=[(dtrain, "train"), (dvalid, "valid")], early_stopping_rounds=20
)


/home/ypochien/yvictor/jane/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:51:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "early_stopping_rounds" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:0.87261	valid-rmse:0.81208
[1]	train-rmse:0.87217	valid-rmse:0.81182
[2]	train-rmse:0.87176	valid-rmse:0.81155
[3]	train-rmse:0.87143	valid-rmse:0.81138
[4]	train-rmse:0.87114	valid-rmse:0.81122
[5]	train-rmse:0.87086	valid-rmse:0.81105
[6]	train-rmse:0.87063	valid-rmse:0.81095
[7]	train-rmse:0.87043	valid-rmse:0.81082
[8]	train-rmse:0.87023	valid-rmse:0.81070
[9]	train-rmse:0.87005	valid-rmse:0.81060
[10]	train-rmse:0.86989	valid-rmse:0.81050
[11]	train-rmse:0.86974	valid-rmse:0.81040
[12]	train-rmse:0.86959	valid-rmse:0.81033
[13]	train-rmse:0.86945	valid-rmse:0.81027
[14]	train-rmse:0.86932	valid-rmse:0.81022
[15]	train-rmse:0.86916	valid-rmse:0.81000
[16]	train-rmse:0.86899	valid-rmse:0.80981
[17]	train-rmse:0.86888	valid-rmse:0.80973
[18]	train-rmse:0.86877	valid-rmse:0.80969
[19]	train-rmse:0.86865	valid-rmse:0.80954
[20]	train-rmse:0.86854	valid-rmse:0.80947
[21]	train-rmse:0.86841	valid-rmse:0.80943
[22]	train-rmse:0.86829	valid-rmse:0.80935
[23]	train-rmse:0.868

In [22]:
model.best_iteration

69

In [23]:
pred_valid = model.predict(dvalid)
pred_valid

array([ 0.01153709,  0.16565733,  0.12176925, ..., -0.10143218,
       -0.07436967, -0.09300931], dtype=float32)

In [25]:
model.save_model("model_v1.xgb")

/home/ypochien/yvictor/jane/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:56:08] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [27]:
! ls -alh *.xgb

-rw-rw-r-- 1 ypochien ypochien  30K Oct 17 16:17 model_v0.xgb
-rw-rw-r-- 1 ypochien ypochien 249K Oct 17 16:56 model_v1.xgb


In [17]:
model = xgb.Booster(model_file="model_v0.xgb")

In [24]:
plan_valid_set.select(
    pl.col("responder_6").alias("target"),
    pl.col("weight").alias("w"),
    pl.Series("pred", pred_valid),
).select(
    (1
    - ((pl.col("target") - pl.col("pred")) ** 2 * pl.col("w")).sum()
    / ((pl.col("target") ** 2 * pl.col("w")).sum())).alias("r2"),
).collect()


r2
f32
0.009541


In [18]:
import jane.kaggle_evaluation.jane_street_inference_server

In [19]:
lags_ : pl.DataFrame | None = None

In [25]:
feature_names = plan_train_feats.collect_schema().names()
# feature_names

In [53]:
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame:
    print(test)
    print(lags)
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_, feature_names, model
    if lags is not None:
        lags_ = lags
    # dtest = xgb.DMatrix(test.select(pl.col("^feature_.*$")))
    dtest = xgb.DMatrix(test.select(pl.col(feature_names)), feature_names=feature_names)
    pred = model.predict(dtest)
    print(pred)
    predictions = test.select(
        'row_id',
        pl.Series("responder_6", pred),
        # pl.lit(0.0).alias('responder_6'),
    )

    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame)
    # with columns 'row_id', 'responer_6'
    assert predictions.columns == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

In [68]:
inference_server = jane.kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

In [69]:
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            'data/test.parquet',
            'data/lags.parquet',
        )
    )

In [65]:
inference_server.run_local_gateway(
    (
        'data/test.parquet',
        'data/lags.parquet',
    )
)

shape: (39, 85)
┌────────┬─────────┬─────────┬───────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ row_id ┆ date_id ┆ time_id ┆ symbol_id ┆ … ┆ feature_75 ┆ feature_76 ┆ feature_77 ┆ feature_78 │
│ ---    ┆ ---     ┆ ---     ┆ ---       ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ i64    ┆ i16     ┆ i16     ┆ i8        ┆   ┆ f32        ┆ f32        ┆ f32        ┆ f32        │
╞════════╪═════════╪═════════╪═══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 0      ┆ 0       ┆ 0       ┆ 0         ┆ … ┆ 0.0        ┆ 0.0        ┆ -0.0       ┆ -0.0       │
│ 1      ┆ 0       ┆ 0       ┆ 1         ┆ … ┆ 0.0        ┆ 0.0        ┆ 0.0        ┆ 0.0        │
│ 2      ┆ 0       ┆ 0       ┆ 2         ┆ … ┆ 0.0        ┆ 0.0        ┆ -0.0       ┆ -0.0       │
│ 3      ┆ 0       ┆ 0       ┆ 3         ┆ … ┆ 0.0        ┆ 0.0        ┆ -0.0       ┆ -0.0       │
│ 4      ┆ 0       ┆ 0       ┆ 4         ┆ … ┆ 0.0        ┆ 0.0        ┆ 0.0        ┆ 0.0    

In [70]:
pl.read_parquet("submission.parquet")

row_id,responder_6
i64,f32
0,0.037667
1,0.037667
2,0.037667
3,0.037667
4,0.037667
…,…
34,0.037667
35,0.037667
36,0.037667


In [72]:
xgb.__version__

'2.1.1'